# Viewing TLEs

This notebook prototypes datashading of TLEs with interactive clicking to show precomputed satellite track for a given TLE:

In [ ]:
import pandas as pd
import numpy as np
import time, datetime, calendar
from colorcet import kbc
import panel as pn
import datetime as dt
import holoviews as hv
from tables import open_file
from holoviews.operation.datashader import rasterize
import skyfield
from skyfield.framelib import itrs
from skyfield.sgp4lib import EarthSatellite
from spatialpandas.geometry import PointArray
from spatialpandas import GeoDataFrame

hv.extension('bokeh')

## Loading TLES

In [ ]:
tle = pd.read_csv('../tle2017.csv')

In [ ]:
print('Example TLE record:\n%r' % tle.iloc[0]['tle'])

## Filtering by available tracks

In [ ]:
computed = open_file("../precomp2.h5", mode='r')
sat_group = computed.get_node("/sat")
num1, num2 = 205, 320
svals = [int(el[1:]) for el in dir(sat_group) if el.startswith('s')]
tle = tle[tle['norad_id'].isin(svals)]

## Functions to handle TLE latitude/longitude 

In [ ]:
def modulo_lon(val):
    if -180 < val < 180:
        return val
    if val < -180:
        return 180 + (val + 180)
    if val > 180:
        return -180 + (val - 180)
    
def compute_lat_lon(line1, line2):
        """
        Get the Lat/Long at the tle epoch  
        """
        sat = EarthSatellite(line1, line2)
        lat, long, _ = sat.at(sat.epoch).frame_latlon(itrs)
        # return the times, lats, longs, and distances in the units specified
        return lat.degrees, long.degrees

def lat_lon_from_lines(lines, abs_max_lat=84):
    """Computes latitude/longitude and a mask to 
    filter TLEs that don't work for Web Mercator
    (abs > 84 degrees by default)""" 
    lons, lats, mask, inner_mask = [], [], [], []
    for line1, line2 in lines:
        lat, lon = compute_lat_lon(line1,line2)
        if None not in [lon, lat]:
            inner_mask.append(abs(lat) < abs_max_lat)
            mask.append(abs(lat) < abs_max_lat)
            lons.append(lon if lon < 180 else (lon - 360))
            lats.append(lat)
        else:
            mask.append(False)
    return np.array(lats)[inner_mask], np.array(lons)[inner_mask], mask

## Computing TLE lat/lons

In [ ]:
new_lines = [el.replace('None\n', '').split('\n')[:2] for el in tle['tle']] # Splitting the file
lats, lons, mask = lat_lon_from_lines(new_lines) # Can take a while to run...

## Computing easting/northings and building spatial dataframe

In [ ]:
eastings, northings = hv.util.transform.lon_lat_to_easting_northing(lons, lats)

In [ ]:
# Create spatialpandas spatially indexed DataFrame (can take a while to construct)
sdf = GeoDataFrame({'geometry':PointArray((lons, lats)),
                    'eastings':eastings,
                    'northings':northings,
                    'norad_id':tle['norad_id'][mask],
                    'epoch_year': tle['epoch_year'][mask],
                    'epoch_day': tle['epoch_day'][mask],
                    })

## Functions to get track around TLE

In [ ]:
def get_track_around_TLE(sat_id, epoch_year, epoch_day, delta_seconds=4*60*60):
    tle_datetime = (dt.datetime(year=epoch_year, month=1, day=1)
                    + dt.timedelta(days=epoch_day-1))
    return get_precomputed_tracks(sat_id, 
                                  tle_datetime-dt.timedelta(seconds=delta_seconds),
                                  tle_datetime+dt.timedelta(seconds=delta_seconds))

def get_precomputed_tracks(satellite, start, end):
    name = "s" + str(satellite)    
    dataz = getattr(sat_group, name)[:]
    start_index = np.searchsorted(dataz[0, :], start.timestamp())
    end_index   = np.searchsorted(dataz[0, :], end.timestamp())
    return dataz[:, start_index: end_index]    

def get_track(track):
    lat, lon = track[1,:], track[2,:]
    lon = np.array([modulo_lon(el) for el in lon])
    eastings, northings = hv.util.transform.lon_lat_to_easting_northing(lon,lat)
    # Heuristic to insert NaNs to break up Curve (prevent wrapping issues at date line)
    inds = np.where(np.diff(eastings) < -2e7)[0] # Big delta to split on
    inds += 1
    eastings = np.insert(eastings, inds, [float('nan') for i in range(len(inds))])
    northings = np.insert(northings, inds, [float('nan') for i in range(len(inds))])
    return hv.Curve((eastings, northings))

## Test `get_track` function for sat ids 205 and 320

In [ ]:
epoch_year, epoch_day = tle.iloc[0]['epoch_year'], tle.iloc[0]['epoch_day']
(get_track(get_track_around_TLE(205, epoch_year, epoch_day, delta_seconds=4*60*60)) 
 + get_track(get_track_around_TLE(320, epoch_year, epoch_day, delta_seconds=4*60*60)))

## DynamicMap callback

In [ ]:
DELTA_SECONDS = 60*60 # Track length in time (seconds)
def mark_track(x,y):
    delta=0.1
    empty = hv.Curve([(0,0)]).opts(alpha=0)
    if None not in [x,y]:
        x, y = hv.util.transform.easting_northing_to_lon_lat(x, y)
        row = sdf.cx[x-delta:x+delta, y-1:y+1]
        if len(row) == 0:
            return empty
        satid = int(row.iloc[0]['norad_id'])
        epoch_year = row.iloc[0]['epoch_year']
        epoch_day = row.iloc[0]['epoch_day']
        track = get_track_around_TLE(satid, epoch_year, epoch_day,
                                              delta_seconds=DELTA_SECONDS)
        return get_track(track).opts(color='red', alpha=1)
    else:
        return empty
    
    
tracks = hv.DynamicMap(mark_track, streams=[hv.streams.Tap()])

In [ ]:
(hv.element.tiles.ESRI().opts(alpha=0.8, bgcolor='black') 
 * rasterize(hv.Points(zip(eastings,northings))).opts(width=900, height=600, cmap=kbc[64:], cnorm='eq_hist')
 * tracks)